# Stable solvers for stiff ODEs


In the previous chapter we introduced explicit Runge-Kutta (ERK) methods and observed
how they could conveniently be implemented as a hierarchy of Python classes. For most
ODE systems, replacing the simple forward Euler method with a higher-order ERK method
will significantly reduce the number of time steps needed to reach a 
specified accuracy. In most cases it will also lead to a reduced overall computation time,
since the additional cost for each time step is more than outweighed by the reduced number 
of steps. However, for a certain class of ODEs we may observe that all the ERK methods require
very small time steps, and any attempt to increase the time step will lead to spurious
oscillations and possible divergence of the solution. These ODE systems are usually
referred to as *stiff*, and none of the explicit methods introduced in the previous
chapters do a very good job at solving them. We shall see that implicit solvers such as
implicit Runge-Kutta (IRK) methods are far better suited for stiff problems, and may give
substantial reduction of the computation time for challenging problems.

# Stiff ODE systems and stability
<div id="sec:stab_analysis"></div>
One very famous example of a stiff ODE system is the Van der Pol equation, which can be 
written as an initial value problem on the form

<!-- Equation labels as ordinary links -->
<div id="vdp1"></div>

$$
\begin{equation}
y_1' = y_2, \quad y_1(0) = 1, \label{vdp1} \tag{1}
\end{equation}
$$

<!-- Equation labels as ordinary links -->
<div id="vdp2"></div>

$$
\begin{equation} 
y_2' = \mu(1-y_1^2)y_2 - y_1, \quad  y_2(0) = 0. \label{vdp2} \tag{2}
\end{equation}
$$

The parameter $\mu$ is a constant which determines the properties of the system, including 
its "stiffness". For $\mu=0$ the problem is a simple oscillator with analytical solution
$y_1 = \cos(t),y_2=\sin(t)$, while for non-zero values of $\mu$ the solution shows
far more complex behavior. The following code implements this system and solves it with the
`ForwardEuler` subclass of the `ODESolver` class hierarchy.

In [1]:
%matplotlib inline

from ODESolver import *
import numpy as np
import matplotlib.pyplot as plt

class VanderPol:
    def __init__(self,mu):
        self.mu = mu

    def __call__(self,t,u):
        du1 = u[1]
        du2 = self.mu*(1-u[0]**2)*u[1]-u[0]
        return du1,du2

model = VanderPol(mu=1)

solver = ForwardEuler(model)
solver.set_initial_condition([1,0])

t,u  = solver.solve(t_span=(0,20),N=1000)

plt.plot(t,u)
plt.show()

[Figure](#fig:vanderpol) shows the solutions for $\mu=0,1$ and 5. Setting $\mu$ even higher, for instance
$\mu=50$, leads to a divergent (unstable) solution with the time step used here ($\Delta t = 0.02$). 
Replacing the FE method with one of the more
accurate ERK method may help a little, but not much. It does help to reduce the time step dramatically, 
but the resulting computation time may be substantial. The time step for this problem is dictated by 
stability requirements rather than our desired accuracy, and there may be significant gains 
from choosing a solver that is more stable than the ERK methods considered so far. 

<!-- dom:FIGURE: [./figs_ch3/vanderpol1.png, width=600 frac=1] Solutions of the Van der Pol model for different values of $\mu$. <div id="fig:vanderpol"></div> -->
<!-- begin figure -->
<div id="fig:vanderpol"></div>

<p>Solutions of the Van der Pol model for different values of $\mu$.</p>
<img src="./figs_ch3/vanderpol1.png" width=600>

<!-- end figure -->


Why does the solution of the Van der Pol model fail so badly for large values of $\mu$? And, more generally,
what are the properties of an ODE system that makes it stiff? To answer these questions, it is useful to 
start with a simpler problem than the Van der Pol model.  
Consider, for instance, a simple IVP known as the Dahlquist test equation;

<!-- Equation labels as ordinary links -->
<div id="linear"></div>

$$
\begin{equation}
u' = \lambda u, \quad u(0) = 1,  \label{linear} \tag{3} 
\end{equation}
$$

where $\lambda$ may be a complex number. With $\lambda = 1$ this is the simple exponential growth 
problem considered earlier, but in this chapter we are primarily interested in $\lambda$ with negative
real part, i.e., either real or complex $\lambda$ that satisfy $\Re(\lambda) < 0$. 
In such cases the solution of ([3](#linear)) decays over time and is completely stable, 
but we shall see that the numerical solutions do not always retain this property.

Following the definition in [[AscherPetzold]](#AscherPetzold), we say that problem ([3](#linear)) is stiff for an interval
$[0,b]$ if the real part of $\lambda$ satisfies
\[
b\Re(\lambda) \ll -1 .
\]
For more general non-linear
problems, such as the Van der Pol model in ([1](#vdp1))-([2](#vdp2)), the
system's stiffness is characterized by the eigenvalues $\lambda_i$ of the local
Jacobian matrix $J$ of the right-hand side function $f$. The Jacobian is defined by 
\[
J_{ij} = \frac{\partial f_i (t,y)}{\partial y_j} ,
\]
and the problem is stiff for an interval $[0,b]$ if
\[
b\min_{i}\Re(\lambda_i) \ll -1.
\]
In the ODE literature one will also find more pragmatic definitions
of stiffness, for instance that an equation is stiff if the 
time step needed to maintain stability of an explicit
method is much smaller than the time step dictated by the accuracy
requirements [[Curtiss52;@AscherPetzold]](#Curtiss52;@AscherPetzold). These definitions indicate that
the stiffness of a problem is not only a function of the ODE itself,
but also of the interval of integration and of the chosen accuracy requirement. A
detailed discussion of stiff ODE systems can be found in, for instance,
[[AscherPetzold;@ODEII]](#AscherPetzold;@ODEII).

Eq. ([3](#linear)) is the foundation for *linear stability analysis*, which is a very
useful technique for analyzing and understanding the stability of ODE solvers. 
The solution to the equation is $u(t) = e^{\lambda t}$, which obviously grows
very rapidly if $\lambda$ with a positive real part. We are therefore primarily 
interested in the case $\Re(\lambda) < 0$, for which the analytical solution is stable, but 
our choice of solver may introduce *numerical instabilities*. The
Forward Euler method applied to ([3](#linear)) gives the update formula

$$
u_{n+1} = u_n +\Delta t \lambda u_n = u_n (1+\Delta t \lambda) ,
$$

and for the first step, since we have $u(0) = 1$, we have

<!-- Equation labels as ordinary links -->
<div id="linear_euler1"></div>

$$
\begin{equation}
    u_1 = u(\Delta t) =  1+\Delta t \lambda . \label{linear_euler1} \tag{4}
\end{equation}
$$

The analytical solution decays exponentially for $\Re(\lambda) < 0$, and it is natural to 
require the same behavior of the numerical solution, and this gives the requirement
that $| 1+\Delta t \lambda | \leq 1$. If $\lambda$ is real and negative, the time
step must be chosen to satisfy $\Delta t \leq -2/\lambda$ to ensure stability. Keep in mind that
this criterion does not necessarily give a very accurate solution, and it may even oscillate and
look completely different from the exact solution. However, choosing $\Delta t$ to satisfy 
the stability criterion ensures that the solution, as well as any spurious oscillations 
and other numerical artefacts, decay with time.

We have observed that the right-hand side of ([4](#linear_euler1)) contains critical information
about the stability of the FE method. This expression is often called the 
*stability function* or the *amplification factor* of the method, and is written on the general form

$$
R(z) = 1+z .
$$

The FE method is stable for all values $\lambda \Delta t$ which give $\|R(\lambda \Delta t)\| <1$,
and this region of $\lambda \Delta t$ values in the complex plane is referred to as the method's
*region of absolute stability*, or simply its *stability region*. 
The stability region for the FE method is shown in the left panel of [Figure](#fig:stab_erk). The stability
domain is a circle with center $(-1,1)$ and radius one. Obviously, 
if $\lambda \ll 0$, require $\lambda \Delta t$ to lie within this circle is quite restrictive for the choice of $\Delta t$. 

<!-- dom:FIGURE: [./figs_ch3/stab_region_erk.png, width=800 frac=1] Stability regions for explicit Runge-Kutta methods. From left: forward Euler, explicit midpoint, and the fourth order method given by ([rk4_0](#rk4_0))-([rk4_5](#rk4_5)). <div id="fig:stab_erk"></div> -->
<!-- begin figure -->
<div id="fig:stab_erk"></div>

<p>Stability regions for explicit Runge-Kutta methods. From left: forward Euler, explicit midpoint, and the fourth order method given by ([rk4_0](#rk4_0))-([rk4_5](#rk4_5)).</p>
<img src="./figs_ch3/stab_region_erk.png" width=800>

<!-- end figure -->


We can easily extend the linear stability analysis to the other explicit RK methods introduced in Chapter 2.
For instance, applying a single step of the explicit midpoint method given by ([midpoint0](#midpoint0))-([midpoint2](#midpoint2)) to 
([3](#linear)) gives

$$
u(\Delta t)  = 1 + \lambda\Delta t + \frac{(\Delta t\lambda)^2}{2} ,
$$

and we identify the stability function for this method as

$$
R(z) = 1 + z + \frac{z^2}{2}.
$$

The corresponding stability region is shown in the middle panel of [Figure](#fig:stab_erk). For the 
fourth order RK method defined in ([rk4_0](#rk4_0))-([rk4_5](#rk4_5)), the same steps reveal that the stability function is

$$
R(z) = 1 + z + \frac{z^2}{2} + \frac{z^3}{6} + \frac{z^4}{24},
$$

and the stability region is shown in the right panel of [Figure](#fig:stab_erk). We observe
that the stability regions of the two higher-order RK methods are larger than that of the FE method, but not much. In fact, 
if we consider the computational cost of each time step for these methods, the FE method is usually superior 
for problems where the time step is governed by stability. 

It can be shown that the stability function
for an $s$-stage explicit RK method is always a polynomial of degree $\leq s$, and it can easily be verified that
the stability region defined by such a polynomial will never be very large. To obtain a significant 
improvement of this situation, we typically need to replace the explicit methods considered so far with implicit RK methods.

# Implicit methods for stability
Since ([3](#linear)) is stable for all values of $\lambda$ with a negative real part, it is natural to look
for numerical methods with the same property. This means that the stability domain for the method 
covers the entire left half of the complex plane, or that its stability function $|R(z)|\leq 1$ whenever $\Re(z) \leq 0$. 
This property is called *A-stability*. As noted above, the stability 
function of an explicit RK method is always a polynomial, and no polynomial satisfies $|R(z)|< 1$ for all $z<0$. 
Therefore, there are no A-stable explicit RK methods. 
An even  stronger stability requirement can be motivated 
by the fact that for $\lambda \ll 0$, the solution to ([3](#linear)) decays very rapidly. It is natural to expect the same
behavior of the numerical solution, by requiring $|R(z)|\rightarrow 0$ as $z\rightarrow -\infty$. This property is 
referred to as *stiff decay*, and an A-stable method that also has stiff decay is called an *L-stable* method. 

The simplest implicit RK method is the backward Euler (BE) method, which can be derived in exactly the same way 
as the FE method, by approximating the derivative with a simple finite difference. The only difference
from the FE method is that the right-hand side is evaluated at step $n+1$ rather than step $n$. 
For a general ODE, we have

$$
\frac{u_{n+1}-u_n}{\Delta t} = f(t_{n+1},u_{n+1}),
$$

and if we rearrange the terms we get

<!-- Equation labels as ordinary links -->
<div id="be_nonlin0"></div>

$$
\begin{equation}
u_{n+1} - \Delta t f(t_{n+1},u_{n+1}) = u_n  .  \label{be_nonlin0} \tag{5}
\end{equation}
$$

Although the derivation is very similar to the FE method, there is a 
fundamental difference in that the unknown $u_{n+1}$ occurs as an argument in the right-hand 
side function $f(t,u)$. Therefore, for nonlinear $f$, ([5](#be_nonlin0)) is a 
nonlinear algebraic equation that must be solved for the unknown $u_{n+1}$, 
instead of the explicit update formula we had for the FE method. This requirement 
makes implicit methods more complex to implement than explicit methods, and they tend to 
require far more computations per time step. Still, as we will demonstrate later, the
superior stability properties still make implicit solvers better suited for stiff problems. 

We will consider the implementation of implicit solvers 
in the section [Implementing implicit Runge-Kutta methods](#sec:irk_simple_impl) below, but
let us first study the stability of the BE method and other implicit RK solvers using the
linear stability analysis introduced above. Applying the 
BE method to ([3](#linear)) yields

$$
u_{n+1} (1-\Delta t\lambda) = u_n,
$$

and for the first time step, with $u(0) = 1$, we get

$$
u_1 = \frac{1}{1-\Delta t\lambda} .
$$

The stability function of the BE method is therefore $R(z) = 1/(1-z)$, and the corresponding stability domain is 
shown in the left panel of [Figure](#fig:stab_irk0). The method is stable for all choices of $\lambda \Delta t$
*outside* the circle with radius one and center at (1,0) in the complex plane, confirming that the BE 
is a very stable method. It is A-stable, since the stability domain covers the entire left half of the
complex plane, and it is also L-stable since the stability function satisfies 
$R(z) \rightarrow 0$ as $\Re(z) \rightarrow -\infty$.

<!-- dom:FIGURE: [./figs_ch3/stab_region_irk0.png, width=800 frac=1] Stability regions for the backward Euler method (left) and the implicit midpoint method and trapezoidal method (right).  <div id="fig:stab_irk0"></div> -->
<!-- begin figure -->
<div id="fig:stab_irk0"></div>

<p>Stability regions for the backward Euler method (left) and the implicit midpoint method and trapezoidal method (right).</p>
<img src="./figs_ch3/stab_region_irk0.png" width=800>

<!-- end figure -->


The BE method fits into the general RK framework defined by ([genrk0](#genrk0))-([genrk1](#genrk1)) in Chapter 
2, with a single stage ($s=1$), and $a_{11} = b_1 = c_1 = 1$. 
As for the FE method considered in Chapter 2, we can reformulate the method slightly
to introduce a stage derivative and make it obvious that the BE method is part of the RK family:

<!-- Equation labels as ordinary links -->
<div id="backward_euler0"></div>

$$
\begin{equation}
    k_1 = f(t_n+\Delta t,u_n+\Delta t k_1), \label{backward_euler0} \tag{6}
\end{equation}
$$

<!-- Equation labels as ordinary links -->
<div id="backward_euler1"></div> <div id="imp_midpoint0"></div>

$$
\begin{equation} 
    u_{n+1} = u_n + \Delta t k_1 . \label{backward_euler1} \tag{7}
\end{equation}
idx{midpoint method !implicit} idx{trapezoidal method ! implicit} idx{Crank-Nicolson method}
The explicit midpoint and
trapezoidal methods mentioned above also have their implicit counterparts. The implicit midpoint method
is given by
\begin{equation}
    k_1 = f(t_n+\Delta t/2,u_n + k_1 \Delta t/2), \label{imp_midpoint0} \tag{8} 
\end{equation}
$$

<!-- Equation labels as ordinary links -->
<div id="imp_midpoint1"></div>

$$
\begin{equation} 
    u_{n+1} = u_n + \Delta t k_1, \label{imp_midpoint1} \tag{9}
\end{equation}
$$

while the implicit trapezoidal rule, or Crank-Nicolson method, is given by

<!-- Equation labels as ordinary links -->
<div id="cn_0"></div>

$$
\begin{equation}
k_1 = f(t_n,u_n), \label{cn_0} \tag{10}
\end{equation}
$$

<!-- Equation labels as ordinary links -->
<div id="cn_1"></div>

$$
\begin{equation} 
k_2 = f(t_n+\Delta t,u_n+\Delta t k_2), \label{cn_1} \tag{11}
\end{equation}
$$

<!-- Equation labels as ordinary links -->
<div id="cn_2"></div>

$$
\begin{equation} 
u_{n+1} = u_n + \frac{\Delta t}{2} (k_1 +k_2) . \label{cn_2} \tag{12}
\end{equation}
$$

Note that this formulation of the Crank-Nicolson is not very common, and can be simplified
considerably by eliminating the stage derivatives and defining the method in terms
of $u_n$ and $u_{n+1}$. However, the formulation in ([10](#cn_0))-([12](#cn_2)) is convenient
for highlighting that it is in fact an implicit RK method. 
The implicit nature of the simple methods above is apparent from the formulas; one of the stage derivatives 
must be computed by solving an equation involving the non-linear function $f$ rather than from an 
explicit update formula. The Butcher tableaus of the three methods are given by

<!-- Equation labels as ordinary links -->
<div id="irk_butcher0"></div>

$$
\begin{equation}
\begin{array}{c|c}
1 & 1 \\ \hline
 & 1
\end{array} ,\hspace{0.5cm}
\begin{array}{c|c}
    1/2 & 1/2 \\ \hline
     & 1
    \end{array} ,\hspace{0.5cm}
\begin{array}{c|cc}
    0 & & \\
    1 & 0 & 1 \\ \hline
     & 1/2 & 1/2
    \end{array} ,\hspace{0.5cm}
\label{irk_butcher0} \tag{13}
\end{equation}
$$

from left to right for backward Euler, implicit midpoint and the implicit trapezoidal method. 

The implicit midpoint method is and the implicit trapezoidal method have the
same stability function, given by $R(z) = (2+z)/(2-z)$. The corresponding stability domain covers
the entire left half-plane of the complex plane, shown in the right panel of [Figure](#fig:stab_irk0).
Both the implicit midpoint method and the trapezoidal method are therefore A-stable methods. However,
we have $R(z)\rightarrow 1$ as $z\rightarrow -\infty$, so the methods do not have stiff decay 
and are therefore not L-stable. In general, the stability functions of implicit RK methods are always
rational functions, i.e., given by

$$
R(z) = \frac{P(z)}{Q(z)},
$$

where $P, Q$ are polynomials of degree at most $s$. (Recall from the section [Stiff ODE systems and stability](#sec:stab_analysis) 
above that the stability functions for the explicit methods are always polynomials of degree at most $s$.) 

The accuracy of the implicit methods considered above can easily be calculated using a Taylor series expansion 
as outlined in the section [sec:error_taylor](#sec:error_taylor), and confirms that the backward Euler method is 
first order accurate while the two others are second order methods. We mentioned
above that an explicit Runge-Kutta method with $s$ stages has order $p\leq s$, but with implicit
methods we have greater freedom in choosing the coefficients $a_{ij}$ and therefore potentially 
higher accuracy for a given number of stages. In fact, the maximum order for an implicit RK method
is $p=2s$, which is precisely the case for the implicit midpoint method, having $s=1$ and $p=2$. 
We will consider more advanced implicit RK methods later, but let us first have a look at
how we can implement the methods introduced so far. 

# Implementing implicit Runge-Kutta methods
<div id="sec:irk_simple_impl"></div>
In the previous section we have demonstrated the superior stability of the implicit methods,
and also mentioned that the accuracy is generally higher, for a fixed number of stages. 
So why are not implicit Runge-Kutta solvers the natural choice for all ODE problem? The
answer to this question is of course the fact that they are implicit, so the stage derivatives
are defined in terms of non-linear equations rather than explicit formulae. This fact complicates
the implementation of the methods and makes each time step far more computationally
expensive. Because of the latter, explicit solvers are usually more efficient for all non-stiff
problems, and implicit solvers should only be used for stiff ODEs. 

For scalar ODEs, solving an equation such as ([5](#be_nonlin0)) or ([8](#imp_midpoint0)) with Newton's method
is usually not  very challenging. However, we are most interested in solving systems of ODEs, 
which complicates the task a bit, since we then need to solve a system of coupled non-linear
equations. Applying Newton's method to a system of equations requires the solution of a system
of linear equations for every iteration, and we are left with a wide variety of choices for how 
to solve these, as well as other solver choices and parameters that
may be tuned to optimize the performance. In the present text we focus on understanding the fundamental
ideas of the IRK solvers, and we will not dive into the details of optimizing the performance. We will therefore
base our implementation on built-in equation solvers from SciPy. We will start with the 
backward Euler method, being the simplest, but we will  
keep the implementation sufficiently general to be easily extended to more advanced implicit 
methods. The interested reader may refer to, for instance, [[AscherPetzold;@ODEII]](#AscherPetzold;@ODEII) for 
a detailed discussion of solver optimization and choices to improve the computational performance.

If we examine the `ODESolver` class first introduced in Chapter 2, we may 
observe that many of the administrative tasks involved in the RK methods are the same for 
implicit as for explicit methods. In particular, the initialization of the solution arrays
and the for-loop that advances the solution are exactly the same, but advancing the 
solution from one step to the next is quite different. It is therefore convenient to implement the implicit solvers
as part of the existing class hierarchy, and let the `ODESolver` superclass handle the 
tasks of initializing the solver as well as the main solver loop. The different explicit 
methods introduced in Chapter 2 where then realized through 
different implementations of the `advance` method. We can use the same approach 
for implicit methods, but since each step involves a few more operations it is
convenient to introduce a couple of additional methods. For instance, 
a compact implementation of the backward Euler method may look as follows:

In [2]:
from ODESolver import *
from scipy.optimize import root

class BackwardEuler(ODESolver):

    def stage_eq(self,k):
        u, f, n, t = self.u, self.f, self.n, self.t
        dt = self.dt
        return k - f(t[n]+dt,u[n]+dt*k)

    def solve_stage(self):
        u, f, n, t = self.u, self.f, self.n, self.t
        k0 = f(t[n],u[n])
        sol = root(self.stage_eq,k0)
        return sol.x

    def advance(self):
        u, f, n, t = self.u, self.f, self.n, self.t
        dt = self.dt
        k1 = self.solve_stage()
        return u[n]+dt*k1

Compared with the explicit solvers presented in Chapter 2 
we have introduced two additional methods in our `BackwardEuler` class. 
The first of these, `stage_eq(self,k)`, is simply a Python
implementation of ([6](#backward_euler0)), which defines the non-linear equation for the 
stage derivative. The method takes 
the stage derivative `k` as input and returns the residual of ([6](#backward_euler0)), which makes it
suitable for use with SciPy non-linear equation solvers. The actual solution of the stage derivate equation
is handled in the `solve_stage` method, which first computes an initial guess `k0` for the stage
derivative, and then passes this guess and the function `stage_eq` to SciPy's `root` method for 
solving the equation. The `root` function returns an object of the class `OptimizeResult`, which includes
the solution as an attribute `x`, as well as numerous other attributes containing information about the 
solution process. We refer to the SciPy documentation for further details on the `OptimizeResult` and the
`root` function.

<!-- dom:FIGURE: [./figs_ch3/vanderpol_be.png, width=600 frac=1] Solutions of the Van der Pol model for $\mu=10$, using the forward and backward Euler methods with $\Delta t = 0.04$. <div id="fig:vdp2"></div> -->
<!-- begin figure -->
<div id="fig:vdp2"></div>

<p>Solutions of the Van der Pol model for $\mu=10$, using the forward and backward Euler methods with $\Delta t = 0.04$.</p>
<img src="./figs_ch3/vanderpol_be.png" width=600>

<!-- end figure -->


We can demonstrate the superior stability of the backward Euler method by returning to the 
Van der Pol equation considered above. Setting, for instance, $\mu=10$, and solving the model with 
both the forward and backward Euler method gives the plots shown in [Figure](#fig:vdp2). The top panel shows
a reference solution computed with the SciPy `solve_ivp` solver and very low tolerance (`rtol=1e-10`).
The middle panel shows the solution produced by forward Euler with $\Delta t = 0.04$, showing visible oscillations in 
one of the solution components. Increasing the time step further leads to a divergent solution. The lower 
panel shows the solution from backward Euler with $\Delta t = 0.04$, which is obviously a lot more stable, 
but still quite different from the reference solution in the top panel. With the backward Euler method, 
increasing the time step further will still give a stable solution, but it does not look like the
exact solution at all. This little experiment illustrates the need to consider both accuracy and stability 
when solving challenging ODEs systems.

Just as we did for the explicit methods in Chapter 2, it is possible to reuse code from 
the `BackwardEuler` class to implement other solvers. Extensive code reuse for a large group of implicit
solvers requires a small rewrite of the code above to a more general form, which will be presented in the next 
section. However, we may observe that a simple solver like the Crank-Nicolson method can be realized as 
a very small modification of our `BackwardEuler` class. A class implementation may look like

In [3]:
class CrankNicolson(BackwardEuler):
    def advance(self):
        u, f, n, t = self.u, self.f, self.n, self.t
        dt = self.dt
        k1 = f(t[n],u[n])
        k2 = self.solve_stage()
        return u[n]+dt/2*(k1+k2)

Here, we utilize the fact that the stage $k_1$ in the Crank-Nicolson is explicit and does not require
solving an equation, while the definition of $k_2$ is identical to the definition of 
$k_1$ in the backward Euler method. We can therefore reuse both the `stage_eq` and `solve_stage` methods directly
and only the `advance` method needs to be reimplemented. This compact implementation of the Crank-Nicolson method
is convenient for code reuse, but it may be argued that it violates a common principle of object-oriented programming.
Subclassing and inheritance is considered an "is-a" relationship, so this class implementation implies that an instance
of the `Crank-Nicolson` class is also an instance of the `BackwardEuler` class. While this works fine in the program, 
and is convenient for code reuse, it is not a correct representation of the relationship between the two 
numerical methods. The Crank-Nicolson method is not a special case of the backward Euler, but, as noted above, 
both methods belong to the group of implicit RK solvers. In the following sections we will 
describe an alternative class hierarchy which is based on this relationship, and enables a compact 
implementation of RK methods by utilizing the general formulation in ([genrk0](#genrk0))-([genrk1](#genrk1))


# Implicit methods of higher order
Just as for the ERK methods considered in Chapter 2, the accuracy of IRK methods
can be increased by adding more stages. However, for implicit methods we have even more
freedom in choosing the parameters $a_{ij}$, and the choice of these impacts both the
accuracy and the computational complexity of the methods. We will here consider two 
main branches of IRK methods; the so-called *fully implicit* methods and the *diagonally implicit* methods. 
Both classes of methods are quite popular and commonly used, and both have their advantages and drawbacks.

## Fully implicit RK methods
 The most general form of RK methods are the fully implicit methods, often referred to as
FIRK methods. These solvers are simply defined by ([genrk0](#genrk0))-([genrk1](#genrk1)), with all
coefficients $a_{ij}$ (potentially) non-zero. For a method with more than one stage, 
this formulation implies that all stage derivatives depend on all other stage derivatives,
so we need to determine them all at once by solving a single system of non-linear 
equations. This operation is quite expensive, but the reward is that the FIRK methods have
superior stability and accuracy for a given number of stages. A FIRK method with 
$s$ stages can have order at most $2s$, which was the case for the implicit midpoint
method in ([8](#imp_midpoint0))-([9](#imp_midpoint1)).

!Many of the most popular FIRK methods are based on combining standard quadrature 
methods for numerical integration with the idea of *collocation*. We present the
basic idea of the derivation here, since many important methods are based on the
same foundation. For the complete details we refer ti, for instance, [[ODEII]](#ODEII).
Recall from Chapter 2 that all Runge-Kutta methods
can be viewed as approximations of ([ode_integral0](#ode_integral0)), where
the integral is approximated by a weighted sum. We set

<!-- Equation labels as ordinary links -->
<div id="ode_integral_approx"></div>

$$
\begin{align*}
    u(t_{n+1}) &= u(t_n) + \int_{t_n}^{t_{n+1}} f(t,u(t)) \approx u(t_n) + \sum_{i=1}^s b_i k_i ,
    \label{ode_integral_approx} \tag{14}
\end{align*}
$$

where $b_i$ are the weights and $k_i$ are the stage derivatives, which 
could be interpreted as approximations of the right-hand side function $f(t,u)$ at distinct 
time points $t_n+\Delta t c_i$. 

Numerical integration is a very well studied branch of 
numerical analysis, and it is natural to choose the integration
points $c_i$ and weights $b_i$ based on standard quadrature rules
with well established properties. Such quadrature rules are often derived
by approximating the integrand with a polynomial which interpolates the function 
$f$ in distinct points, and then integrating the
polynomial exactly. The same idea can be used in the derivation
of implicit RK methods. We approximate the solution 
$u$ on the interval $t_n < t \leq t_{n+1}$ by a polynomial $P(t)$ of 
degree at most $s$, and then require that $P(t)$ solves the ODE exactly in
distinct points $t_n+c_i\Delta t$. This requirement, i.e., that

<!-- Equation labels as ordinary links -->
<div id="colloc0"></div>

$$
\begin{equation}
P'(t_i) = f(t_i,P(t_i)), \quad  t_i = t_n+c_i\Delta t, i = 1,\ldots , s .
\label{colloc0} \tag{15}\end{equation}
$$

is known as collocation, and is a widely used idea in numerical analysis. 
It can be shown that, given a choice of quadrature points $c_i$, 
the collocation equations given by ([15](#colloc0)) determines the remaining method coefficients $a_{ij}$ 
and $b_i$ uniquely, see [[AscherPetzold]](#AscherPetzold) for details. 

A convenient way to 
derive FIRK methods is to choose a set of collocation points $c_i$, typically
chosen from standard quadrature rules, and solve ([15](#colloc0))
to determine the remaining parameters. This approach has led to families of FIRK methods based on 
common rules for numerical integration. For instance, choosing $c_i$ as Gauss points
gives rise to the Gauss methods, which are the most accurate methods for a given number
of stages, having order $2s$.
The single stage Gauss method is the implicit midpoint method introduced above, 
while the fourth order Gauss method with $s=2$ is defined by the Butcher tableau

$$
\begin{array}{c|cc}
    \frac{3-\sqrt{3}}{6} &\frac{1}{4} & \frac{3-2\sqrt{3}}{12}\\
    \frac{3+\sqrt{3}}{6}  & \frac{3+2\sqrt{3}}{12} & \frac{1}{4} \\ \hline
     & \frac{1}{2}& \frac{1}{2}
    \end{array} .
$$

The Gauss methods are A-stable but not L-stable, and since we typically only use
FIRK methods for challenging stiff problems where stability is important, another
family of FIRK methods is more popular in practice. These are the Radau IIA methods, 
which are based on Radau quadrature points which include the right end of the 
integration interval (i.e., $c_s = 1$). The one-stage Radau IIA method is the backward 
Euler method, while two- and three-stage versions are given by

$$
\begin{array}{c|cc}
    1/3 &5/12 & -1/12\\ 
    1 & 3/4 & 1/4 \\ \hline
     & 2/3& 1/4
    \end{array} , \hspace{1.0cm}
    \begin{array}{c|ccc}
        \frac{4-\sqrt{6}}{10} &\frac{88-7\sqrt{6}}{360} & \frac{296-169\sqrt{6}}{1800}&\frac{-2+3\sqrt{6}}{225}\\
        \frac{4+\sqrt{6}}{10} & \frac{296+169\sqrt{6}}{1800} & \frac{88+7\sqrt{6}}{360} & \frac{-2-3\sqrt{6}}{225} \\ 
        1 & \frac{16-\sqrt{6}}{36} & \frac{16+\sqrt{6}}{36} & \frac{1}{9} \\ \hline
        & \frac{16-\sqrt{6}}{36} & \frac{16+\sqrt{6}}{36} & \frac{1}{9} \\ 
        \end{array} .
$$

The Radau IIA methods have order $s-1$, and their stability functions are $(s-1,s)$ Padé approximations
to the exponential function [[ODEII]](#ODEII). For the two- and three-stage methods
above, the stability functions are

$$
\begin{align*}
R(z) &= \frac{1+z/3}{1-2z/3 +z^2/6} ,\\
R(z) &= \frac{1+2z/5+z^2/20}{1-3z/5 +3z^2/20-z^2/60}, 
\end{align*}
$$

respectively, with stability domains shown in [Figure](#fig:stab_radau).
The methods are L-stable, which makes them a popular choice for solving 
stiff ODE systems. However, as noted above, the fact that all $a_{ij} \neq 0$ complicates 
the implementation of the methods and makes each time step computationally expensive. All the
$s$ equations of ([genrk0](#genrk0)) become fully coupled and need to be solved simultaneously. 
For an ODE system consisting of $m$ ODEs, we need to solve a system of $m s$ non-linear
equations for each time step. We will come back the implementation of FIRK methods in 
the section [Implementing higher order IRK methods](#sec:irk_implementation), but let us first introduce a slightly simpler class 
of implicit RK solvers. 


<!-- dom:FIGURE: [./figs_ch3/stab_region_radau.png, width=600 frac=1] The shaded area is the stability region for two of the RadauIIA methods, with $s=2$ (left) and $s=3$ (right). <div id="fig:stab_radau"></div> -->
<!-- begin figure -->
<div id="fig:stab_radau"></div>

<p>The shaded area is the stability region for two of the RadauIIA methods, with $s=2$ (left) and $s=3$ (right).</p>
<img src="./figs_ch3/stab_region_radau.png" width=600>

<!-- end figure -->



## Diagonally implicit RK methods
 Diagonally implicit RK methods, or DIRK methods for short, are also sometimes also 
referred to as semi-explicit methods. For these methods, we have $a_{ij} = 0$ for
all $j>i$. (Notice the small but important difference from the explicit methods,
where we have $a_{ij} = 0$ for $j\geq i$.) The consequence of this
choice is that the equation for a single stage derivative $k_i$ 
does not involve stages $k_{i+1}, k_{i+2}$, and so forth, and we can therefore solve 
for the stage derivatives one by one sequentially. We still need to solve
non-linear equations to determine each $k_i$, but we can solve $s$ systems of $m$
equations rather than one large system to compute all the stages at once. This 
property simplifies the implementation and reduces the computational expense 
per time step. However, as expected, the restriction on the method coefficients 
reduces the accuracy and stability compared with the FIRK methods. 
A general DIRK method with $s$ stages has maximum order $s+1$, and methods optimized for stability 
typically have even lower order. 

From the definition of the DIRK methods, we may observe that the implicit midpoint method
introduced above is, technically, a DIRK method. However, is method is also a fully implicit
Gauss method, and is not commonly referred to as a DIRK method. The distinction between FIRK and DIRK methods
is only meaningful for $s>1$. The
Crank-Nicolson (implicit trapezoidal) method given by ([10](#cn_0))-([12](#cn_2)) is 
also a DIRK method, which is evident from the leftmost Butcher tableau in ([13](#irk_butcher0)). 
These methods are, however, only A-stable, and it is possible to derive DIRK methods with
better stability properties. An example of an L-stable, two-stage DIRK method of order two is given by

<!-- Equation labels as ordinary links -->
<div id="dirk2_v0"></div>

$$
\begin{equation}
\begin{array}{c|cc}
    \gamma &\gamma & 0 \\
    1  & 1-\gamma& \gamma\\ \hline
     & 1-\gamma& \gamma
    \end{array} ,
\label{dirk2_v0} \tag{16}\end{equation}
$$

with $\gamma = 1\pm \sqrt(2)/2$. The stability function is

$$
R(z) = \frac{1+z(1-2\gamma)}{(1-z\gamma)^2},
$$

and are A-stable for $\gamma >1/4$. For $\gamma = 1\pm \sqrt(2)/2$ the method is L-stable and 
second order accurate. Note that 
choosing $\gamma > 1$ means that we estimate the stage derivatives outside the 
interval $(t_n,t_{n+1})$, and for the last step outside the time interval of interest. 
While this does not affect the stability or accuracy of the method it may not make sense
for all ODE problems, and the most popular choice is therefore $\gamma = 1-\sqrt(2)/2$.
Notice also that in this method the two diagonal entries of $a_{ij}$ 
are identical, we have $a_{11}=a_{22}=\gamma$. This choice is very common in DIRK
methods, and methods of this kind are usually referred to as *singly diagonally
implicit* RK (SDIRK) methods. The main benefit of this structure is that 
the non-linear equations for each stage derivative become very similar, which
we can take advantage of when solving the equations with quasi-Newton methods.
In particular, the systems will have the same Jacobian matrix, which can 
then be reused for all stages. 


While we do not aim to present a complete overview of the various sub-categories
of RK methods, one additional class of method is worth mentioning. These are
the so called ESDIRK methods, which are simply SDIRK methods where the first stage
is explicit. The motivation for such methods is that the non-linear algebraic equations
involved in the implicit methods are always solved with iterative methods, which
require an initial guess for the solution. For SDIRK methods, it is convenient to
use the previous stage derivate as initial guess for the next one, which will usually provide 
a good initial guess. This approach is obviously not possible for the first stage, but an 
explicit formula for the first stage solves this problem. The simplest ESDIRK method is the implicit trapezoidal (Crank-Nicolson) method
introduced above, and a popular extension of this method is given by

<!-- Equation labels as ordinary links -->
<div id="butcher-tr-bdf2"></div>

$$
\begin{equation}
\begin{array}{c|ccc}
    0 & 0&  & \\
    2\gamma & \gamma & \gamma & 0\\ 
    1 & \beta & \beta & \gamma \\ \hline
     & \beta & \beta & \gamma \\
    \end{array} ,\hspace{0.5cm} 
    \label{butcher-tr-bdf2} \tag{17}
\end{equation}
$$

with $\gamma = 1-\sqrt{2}/2$ and $\beta = \sqrt{2}/4$. The resulting equations for each time step are

$$
\begin{align*}
k_1 &= f(t_n, u_n),\\ 
k_2 &= f(t_n+2\gamma\Delta t,u_n+\Delta t(\gamma k_1 + \gamma k_2)), \\
k_3 &= f(t_n+\Delta t,u_n+\Delta t(\beta k_1 + \beta k_2+\gamma k_3)), \\
u_{n+1} &= u_n + \Delta t (\beta k_1 +\beta k_2+ \gamma k_3) . 
\end{align*}
$$

This method can be interpreted as the sequential 
application of the trapezoidal method and a popular multistep solver called BDF2 
(*backward differentiation formula* of order 2), and it is commonly referred
to as the TR-BDF2 method. It is second order accurate, just as the trapezoidal rule, but it 
is also L-stable and therefore suitable for stiff problems. 

# Implementing higher order IRK methods
<div id="sec:irk_implementation"></div>
In the section [Implementing implicit Runge-Kutta methods](#sec:irk_simple_impl) we implemented two of the simplest implicit RK methods by a relatively 
small extension of the `ODEsolver` class hierarchy. We could easily continue this idea for the more complex
IRK methods, and all the different methods could be realized by separate implementations of the three methods 
`solve_stage`, `stage_eq`, and `advance`. However, these three methods essentially implement the
equations given by ([genrk0](#genrk0))-([genrk1](#genrk1)), which are common for all RK solvers. It is natural
to look for an implementation that allows even more code reuse between the various methods, and we shall see
that such a general implementation is indeed possible. However, it still makes sense to treat the
fully implicit methods and SDIRK methods separately, since the stage calculations of these two method classes are
fundamentally different. 

## A base class for fully implicit methods
One approach to implement the fully implicit RK methods is to rewrite the 
`solve_stage`, `stage_eq`, and `advance` methods of the 
`BackwardEuler` class  above in a completely general manner, so they can handle any number of stages and any choice of
method parameters $a_{ij},b_i$, and $c_i$. New methods can then be implemented simply by setting the number of stages and 
defining the parameter values. In the methods considered so far all the method coefficients have been 
hard-coded into the mathematical expressions, typically inside the `advance` methods, but with the generic approach
it is natural to define them as class attributes in the constructor. A general base class for implicit RK
methods may look as follows:

In [4]:
class ImplicitRK(ODESolver):
    def solve_stages(self):
        u, f, n, t = self.u, self.f, self.n, self.t
        neq = self.neq
        s = self.stages
        k0 = f(t[n],u[n]) 
        k0 = np.hstack([k0 for i in range(s)])
        
        sol = root(self.stage_eq,k0) 

        return np.split(sol.x,s)

    def stage_eq(self,k_all):
        a,c = self.a, self.c 
        s, neq = self.stages, self.neq

        u, f, n, t = self.u, self.f, self.n, self.t
        dt = self.dt

        res = np.zeros_like(k_all)
        k = np.split(k_all,s)
        for i in range(s):
            fi = f(t[n]+c[i]*dt, 
                   u[n]+dt*sum([a[i,j]*k[j] for j in range(s)]))
            res[i*neq:(i+1)*neq] = k[i] - fi 
                                            
        return res

    def advance(self):
        b = self.b 
        u, n, t = self.u, self.n, self.t
        dt = self.dt
        k = self.solve_stages()
        
        return u[n]+dt*sum(b_*k_ for b_,k_ in zip(b,k))

Note that we assume that the method parameters are assumed to be held in NumPy arrays 
`self.a, self.b, self.c`, which need to be defined in subclasses. The `ImplicitRK` class
is meant to be a pure base class for holding common code, and is not intended to be a usable solver
class in itself. The three methods are generalizations of the
same methods in `BackwardEuler` class, and perform the same tasks, but the abstraction level is higher and 
the methods rely on a bit of NumPy magic: 
* The `solve_stages` method is obviously a generalization of the `solve_stage` method above, and most of the
  lines are quite similar and should be self-explanatory. However, be aware that we are now implementing a general
  IRK method with $s$ stages, and we solve a single system of non-linear equations to determine all $s$ stage
  derivatives at once. The solution of this system is a one-dimensional array of length 
  `self.stages * self.neq`, which contains all the stage derivatives. The line `k0 = np.hstack([k0 for i in range(s)])`
  takes an initial guess `k0` for a single stage, and simply stacks it after itself $s$ times to create the initial guess
  for all the stages, using NumPy's `hstack` function and a list comprehension. 

* The `stage_eq` method is also a pure generalization of the `BackwardEuler` version, and performs the same tasks. 
  The first few lines should be self-explanatory, while the `res = np.zeros_like(k_all)` defines an array of the 
  correct length to hold the residual of the equation. Then, for convenience, the line `k = np.split(k_all,s)` 
  splits the array `k_all` into a list `k` containing the individual stage derivatives, which is used inside
  the for loop on the next four lines. This loop forms the core of the method, and 
  is essentially just ([genrk0](#genrk0)) implemented in Python code, split over several lines for improved readability.  
  The residual is returned as a single array of length `self.stages * self.neq`, as expected by the SciPy `root` function.

* Finally, the `advance` method calls the `solve_stages` to compute all the stage derivatives, and then advances
  the solution using a general implementation of ([genrk1](#genrk1)).

With the general base class at hand, we can easily implement new solvers, simply by writing the constructors
that define the method coefficients. The following code implements the implicit midpoint and the two- and
three-stage Radau methods:

In [5]:
class ImpMidpoint(ImplicitRK):
    def __init__(self,f):
        super().__init__(f)
        self.stages = 1
        self.a = np.array([[1/2]])
        self.c = np.array([1/2])
        self.b = np.array([1])


class Radau2(ImplicitRK):      
    def __init__(self,f):
        super().__init__(f)
        self.stages = 2
        self.a = np.array([[5/12,-1/12],[3/4,1/4]])
        self.c = np.array([1/3,1])
        self.b = np.array([3/4, 1/4])


class Radau3(ImplicitRK):
    def __init__(self,f):
        super().__init__(f)
        self.stages = 3
        sq6 = np.sqrt(6)
        self.a = np.array([ [(88-7*sq6)/360,
                            (296-169*sq6)/1800,
                            (-2+3*sq6)/(225)],
                            [(296+169*sq6)/1800, 
                            (88+7*sq6)/360,
                            (-2-3*sq6)/(225)],
                            [(16-sq6)/36, (16+sq6)/36,1/9]])
        self.c = np.array([(4-sq6)/10,(4+sq6)/10,1])
        self.b = np.array([(16-sq6)/36, (16+sq6)/36,1/9])

Notice that we always define the method coefficients as NumPy arrays, even for the
implicit midpoint method where they all contain a single number. This definition is
necessary for the generic methods of the `ImplicitRK` class to work. 

## Base classes for SDIRK and ESDIRK methods
We could, in principle, implement both the SDIRK and ESDIRK 
methods in the same manner as the FIRK methods above, simply by defining
the method coefficients in the constructor. The generic methods from the
`ImplicitRK` base class will work fine even if we have $a_{ij} = 0$ 
for $j>i$. However, the motivation for deriving diagonally implicit methods is precisely to
avoid solving these large systems of non-linear equations, so it does not make much
sense to implement them in this way. Instead, we should utilize the structure of 
the method coefficients and solve for the stage variables sequentially. 
This requires rewriting the two methods `solve_stages` and `stage_eq` from the
base class above. Once the stage derivatives have been computed, advancing the
solution to the next step occurs in the same way for all RK methods, 
so the `advance` method can be left unchanged. 

Considering first the SDIRK methods, we can implement these as 
subclasses of the `ImplicitRK` class, which
enables some (moderate) code reuse and reflects the fact that SDIRK methods are indeed
special cases of implicit RK methods. Using the two-stage SDIRK method defined by ([16](#dirk2_v0)) as an example, 
we get a better view of the tasks involved in the SDIRK methods if we write out the equations for the
stage derivatives. Inserting the coefficients from ([16](#dirk2_v0)) into 
([genrk0](#genrk0))-([genrk1](#genrk1)) gives

<!-- Equation labels as ordinary links -->
<div id="dirk2_eq0"></div>

$$
\begin{equation}
k_1 = f(t_n+\gamma\Delta t, u_n+\Delta t \gamma k_1), \label{dirk2_eq0} \tag{18}
\end{equation}
$$

<!-- Equation labels as ordinary links -->
<div id="dirk2_eq1"></div>

$$
\begin{equation} 
k_2 = f(t_n+\Delta t,u_n+\Delta t((1-\gamma)k_1 + \gamma k_2)), \label{dirk2_eq1} \tag{19}
\end{equation}
$$

<!-- Equation labels as ordinary links -->
<div id="dirk2_eq2"></div>

$$
\begin{equation} 
u_{n+1} = u_n + \Delta t ((1-\gamma)k_1 +\gamma k_2) . \label{dirk2_eq2} \tag{20}
\end{equation}
$$

Here, ([18](#dirk2_eq0)) is nearly identical to the equation defining the stage derivative
in the backward Euler method, the only difference being the factor $\gamma$ 
in front of the arguments inside the function call. Furthermore, the only difference between ([18](#dirk2_eq0)) and
([19](#dirk2_eq1)) is the additional term $\Delta t(1-\gamma)k_1$ inside the function call. 
In general, any stage equation for any DIRK method can be written as

<!-- Equation labels as ordinary links -->
<div id="sdirk_stage"></div>

$$
\begin{equation}
k_i = f(t_n+c_i\Delta t, u_n+\Delta t(\sum_{j=0}^{i-1}a_{ij}k_j + \gamma k_i)), \label{sdirk_stage} \tag{21}
\end{equation}
$$

where the sum inside the function call only includes previously computed stages. 

Given the similarity of ([21](#sdirk_stage)) with the stage equation from the backward Euler method, 
it is natural to implement the SDIRK stage equation as a generalization of the `stage_eq` 
method from the `BackwardEuler` class. It is also convenient to place this method in an SDIRK
base class, from which we may derive all specific SDIRK solver classes. 
Furthermore, since the stage equations can be written on this general form, it is not difficult
to generalize the algorithm for looping through the stages and computing the individual 
stage derivatives. The base class can, therefore, contain general SDIRK versions of both the `stage_eq`
and `solve_stages`, and the only task left in individual solver classes is to define the number
of stages and the method coefficients. The complete base class implementation may look as follows.

In [6]:
class SDIRK(ImplicitRK):
    def stage_eq(self,k,c_i, k_sum):
        u, f, n, t = self.u, self.f, self.n, self.t
        dt = self.dt
        gamma = self.gamma

        return k - f(t[n]+c_i*dt,u[n]+dt*(k_sum+gamma*k))

    def solve_stages(self):
        u, f, n, t = self.u, self.f, self.n, self.t
        a, c = self.a, self.c    
        s = self.stages

        k = f(t[n],u[n]) #initial guess for first stage
        k_sum = np.zeros_like(k)
        k_all = []
        for i in range(s):
            k_sum = sum(a_*k_ for a_,k_ in zip(a[i,:i],k_all))
            k = root(self.stage_eq,k,args=(c[i],k_sum)).x
            k_all.append(k)
        
        return k_all

The modified `stage_eq` method takes two additional parameters; the coefficient `c_i` corresponding to the current stage, 
and the array `k_sum` which holds the sum $\sum_{j=1}^{i-1}a_{ij}k_j$. These arguments need to be
initialized correctly for each stage, and passed as additional arguments to the SciPy `root` function. 
For convenience, we also assume that the method parameter $\gamma$ has been stored as a separate 
class attribute. With the `stage_eq` method implemented in this general way, the `solve_stages` method 
simply needs to update the weighted sum of previous stages (`k_sum`), and pass this and the correct $c$ value
as additional arguments to the SciPy `root` function. The implementation above implements this in a for loop
which computes the stage derivatives sequentially and returns them as a list `k_all`. 


<!-- dom:FIGURE: [./figs_ch3/vanderpol_irk.png, width=600 frac=1] Solutions of the Van der Pol model for $\mu=10$ and $\Delta t = 0.1$, using implicit RK solvers of different accuracy. <div id="fig:vanderpol_irk"></div> -->
<!-- begin figure -->
<div id="fig:vanderpol_irk"></div>

<p>Solutions of the Van der Pol model for $\mu=10$ and $\Delta t = 0.1$, using implicit RK solvers of different accuracy.</p>
<img src="./figs_ch3/vanderpol_irk.png" width=600>

<!-- end figure -->


As for the FIRK method classes, the only method we now need to implement specifically for each solver class is the
constructor, in which we define the number of stages and the method coefficients. A class implementation of the
method in ([16](#dirk2_v0)) may look as follows.

In [7]:
class SDIRK2(SDIRK):
    def __init__(self,f):
        super().__init__(f)
        self.stages = 2
        gamma = (2-np.sqrt(2))/2
        self.gamma = gamma
        self.a = np.array([[gamma,0],
                            [1-gamma, gamma]])
        self.c = np.array([gamma,1])
        self.b = np.array([1-gamma, gamma])

Shifting our attention to the ESDIRK methods, these are identical to the SDIRK methods except for the
first stage, and the potential for code reuse is obvious. Examining the two methods of the 
SDIRK base class above, we quickly conclude that the `stage_eq` method can be reused in an ESDIRK solver
class, since the equations to be solved for each stage are identical for SDIRK and ESDIRK solvers. 
However, the `solve_stages` method needs to be modified, since there is no need to solve a non-linear
equation for `k1`. The modifications can, however, be very small, since all stages $i>1$ are identical. 
A possible implementation of the ESDIRK class can look as follows:

In [8]:
class ESDIRK(SDIRK):
    def solve_stages(self):
        u, f, n, t = self.u, self.f, self.n, self.t
        a, c = self.a, self.c    
        s = self.stages

        k = f(t[n],u[n]) 
        k_sum = np.zeros_like(k)
        k_all = [k]
        for i in range(1,s):
            k_sum = sum(a_*k_ for a_,k_ in zip(a[i,:i],k_all))
            k = root(self.stage_eq,k,args=(c[i],k_sum)).x
            k_all.append(k)

        return k_all

Comparing with the SDIRK base class above, the two methods look identical at first, but there are two small but important 
differences. The first is that the result of the first function 
evaluation `k = f(t[n],u[n])` is now used directly as the first stage, by setting `k_all = [k]`, 
instead of just serving as an initial guess for the nonlinear equation solver. 
The second is that the for-loop for computing the remaining stages starts at `i=1` rather than `i=0`. 

With the ESDIRK base class at hand, we can implement individual ESDIRK methods simply by defining the
constructor, for instance

        class BDF_TR2(ESDIRK):
            def __init__(self,f):
                super().__init__(f)
                self.stages = 3
                gamma = 1-np.sqrt(2)/2
                beta = np.sqrt(2)/4
                self.gamma = gamma
                self.a = np.array([[0,0,0],
                                    [gamma, gamma,0],
                                    [beta,beta,gamma]])
                self.c = np.array([0,2*gamma,1])
                self.b = np.array([beta,beta,gamma])


Notice that these class implementations have some potential weaknesses. An obvious one is that the `solve_stages` methods
of the SDIRK and ESDIRK classes are nearly identical, and most of the code is duplicated. Part of the purpose
of implementing the solvers in a class hierarchy is to avoid code duplication, so this is obviously not optimal. However, 
avoiding duplicated code completely would in this case require refactoring the classes a bit, to split the
tasks performed in `solve_stages` into several methods. Since these tasks belong quite naturally together, splitting
them up could make the code more difficult to read and understand, and would also potentially make the code 
less computationally efficient. The latter should always be a consideration when implementing numerical methods, 
although it is not a strong focus of the present text. 

Another choice that can be questioned in the `ESDIRK` classis that we retain the 
dimensions of the `self.a` coefficient array, and simply set the entire first row to zero. Storing these zeros is obviously 
not needed, and we could have omitted them and adjusted the for-loop in `solve_stages` accordingly. However, this choice
would make the link between the code and the mathematical formulation of RK methods less obvious, and the benefits would 
be minimal. 

[Figure](#fig:vanderpol_irk) illustrates the difference in accuracy between a number of IRK solvers. The 
chosen time step $\Delta t = 0.1$ is obviously too large for the backward Euler method, 
and the solution is not even close to the reference solution. The other solvers are the three-stage SDIRK method of order
two, the two-stage Radau method of order three, and three-stage Radau method of order five. 
We will see more examples of SDIRK methods in Chapter 4, when we introduce RK methods with adaptive time step.